In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from datetime import date, time, timedelta
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import ignite
from sklearn.metrics import r2_score
import seaborn as sn
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


In [12]:
analysts = "../data/analysts.csv"
df_an = pd.read_csv(analysts)
df_an['STATPERS'] = pd.to_datetime(df_an['STATPERS'], format='%Y%m%d')
df_an['STATPERS'] = df_an['STATPERS'].apply(handle_weekends)
df_an['TGTDATE'] = df_an['STATPERS'].apply(add_year)

data_start_price = price_df.rename(columns={'PRC': 'start_price'})
data_end_price = price_df.rename(columns={'PRC': 'end_price'})

df_an = pd.merge(df_an, data_start_price, how='left', left_on=['STATPERS', 'CUSIP'], right_on=['date', 'CUSIP'])
df_an = pd.merge(df_an, data_end_price, how='left', left_on=['TGTDATE', 'CUSIP'], right_on=['date', 'CUSIP'])
df_an = df_an.drop(['PERMNO_x', 'date_x', 'COMNAM_x','PERMNO_y', 'date_y', 'COMNAM_y'], axis=1).fillna(0)



In [ ]:
df_an = df_an.drop(df_an[df_an['start_price'] <= 0].index)
df_an = df_an.drop(df_an[df_an['end_price'] <= 0].index)
df_an = df_an.drop(df_an[df_an['MEANPTG'] == 0].index)
df_an = df_an[df_an['CUSIP'].isin(companies)]
df_an = df_an[df_an['STATPERS'] <= '2020-11-30']
df_an = df_an[df_an['STATPERS'] >= '2018-06-29']
df_true = df_an.drop(['CUSIP', 'OFTIC', 'CNAME', 'STATPERS', 'NUMEST', 'STDEV', 'TGTDATE'], axis=1)


In [20]:
df_true['TrueGrowth'] = (((df_true['end_price']-df_true['start_price']) / df_true['start_price']))
df_true['PredGrowth'] = (((df_true['MEANPTG']-df_true['start_price']) / df_true['start_price']))

In [35]:
r2 = r2_score(df_true['TrueGrowth'], df_true['PredGrowth'])
print(f"AND THE ANALYSTS' SICK R2 IS: {r2}")


AND THE ANALYSTS' SICK R2 IS: -0.04165983672854545
AND THE ANALYSTS' SICK R2 IS: -16.69115780855374
AND THE ANALYSTS' SICK R2 IS: 0.873637814754343
AND THE ANALYSTS' SICK R2 IS: 0.8247327049585713


In [387]:
accuracy_level = 1 - abs((df['TrueGrowth']/df['PredGrowth'])-1)
print(accuracy_level.describe())
print(accuracy_level.median())


count    74667.000000
mean         0.529265
std          0.950643
min        -73.360000
25%          0.413281
50%          0.672280
75%          0.848753
max          1.000000
dtype: float64
0.6722803668223503
